#  LexiLingo: Unified LoRA Adapter Fine-tuning (Multi-Task Learning)

**Version:** 2.8 (với Auto-Resume Checkpoint Support)

**Mục đích:** Fine-tune Qwen2.5-1.5B-Instruct với **1 unified LoRA adapter** để xử lý đồng thời 4 tasks:
1.  **Fluency Scoring** (0.0-1.0)
2.  **Vocabulary Level Classification** (A1, A2, B1, B2, C1, C2)
3.  **Grammar Error Correction** (GEC)
4.  **Dialogue Generation** (conversational responses)

---

##  **Tính Năng Mới: Auto-Resume Training**

###  Checkpoint được tự động lưu:
-  Mỗi **100 steps** (GPU) hoặc **50 steps** (CPU)
-  Lưu kèm: model weights, optimizer state, scheduler state, training metrics
-  Giữ lại **3 checkpoints** mới nhất (tự động xóa checkpoints cũ)
-  Training state được lưu vào `training_state.json`

###  Cách sử dụng:

#### **Lần đầu training:**
```python
# Chỉ cần run cell training, checkpoint sẽ tự động được lưu
unified_model, trainer = finetune_unified_adapter(
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    lora_config=UNIFIED_LORA_CONFIG,
    resume_from_checkpoint="auto",  # Tự động resume nếu có checkpoint
)
```

#### **Resume sau khi đổi runtime/disconnect:**
1. Mount Drive (nếu lưu trên Drive)
2. Run lại cells setup (1-3)
3. Run cell config & load model
4. Run cell training → **Tự động resume từ checkpoint mới nhất!**

#### **Bắt đầu training mới (bỏ qua checkpoints cũ):**
```python
resume_from_checkpoint=None  # Thay vì "auto"
```

#### **Resume từ checkpoint cụ thể:**
```python
resume_from_checkpoint="/path/to/checkpoint-1000"
```

---

**Optimized for:** Google Colab T4 GPU (15GB VRAM)

## 1. Setup Environment

In [1]:
# Install required packages (Colab-compatible)
# NOTE: protobuf must be >=5.26.1 but <6.0.0 for Colab compatibility
!pip install -q -U \
  "protobuf>=5.26.1,<6.0.0" \
  "pandas==2.2.2" \
  transformers>=4.41.0 \
  accelerate>=0.29.0 \
  datasets>=2.18.0 \
  peft>=0.10.0 \
  trl>=0.9.6 \
  bitsandbytes>=0.43.1 \
  sentencepiece \
  scipy \
  wandb \
  pymongo \
  matplotlib \
  seaborn \
  scikit-learn

In [ ]:
# QUAN TRỌNG: Mount Google Drive để lưu checkpoint và model
# Checkpoint sẽ được lưu vào Drive để không mất khi Colab disconnect
import os
from pathlib import Path

try:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Verify Drive is mounted
    drive_path = Path('/content/drive/MyDrive')
    if drive_path.exists():
        print("\n" + "="*70)
        print("GOOGLE DRIVE MOUNTED SUCCESSFULLY")
        print("="*70)
        print(f"Drive path: {drive_path}")
        print("\nCheckpoint will be saved to:")
        print(f"  /content/drive/MyDrive/LexiLingo/models/unified/")
        print("\nThis ensures data persists even if Colab disconnects!")
        print("="*70 + "\n")
    else:
        print("\nWARNING: Drive mount failed! Checkpoints will be lost on disconnect.")
        
except Exception as e:
    print(f"\nRunning locally (not Colab): {e}")
    print("Checkpoints will be saved to: ./model/outputs/unified/\n")

In [ ]:
# Verify installed versions
import sys
import subprocess
print("Installed versions:")
subprocess.run([sys.executable, "-m", "pip", "show", "protobuf"], check=False)
subprocess.run([sys.executable, "-m", "pip", "show", "pandas"], check=False)
print("\nNote: If you see import errors, restart runtime (Runtime → Restart runtime), then run from Cell 1.")

In [2]:
import torch
import json
import os
from pathlib import Path
from datasets import Dataset, load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    set_seed,
 )
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
 )
from trl import SFTTrainer
import numpy as np

# Reproducibility
set_seed(42)

# Device & precision (Colab GPU-first)
if torch.cuda.is_available():
    device = torch.device('cuda')
    major, minor = torch.cuda.get_device_capability(0)
    use_bf16 = major >= 8  # Ampere+
    use_fp16 = not use_bf16
    print(f" CUDA available: {torch.cuda.get_device_name(0)} (capability {major}.{minor})")
    print(f"Precision: {'bf16' if use_bf16 else 'fp16'}")
elif torch.backends.mps.is_available():
    device = torch.device('mps')
    use_bf16 = False
    use_fp16 = False
    print(" MPS available (Apple Silicon)")
else:
    device = torch.device('cpu')
    use_bf16 = False
    use_fp16 = False
    print(" Running on CPU (slow). Consider Colab GPU.")

print(f"PyTorch version: {torch.__version__}")

In [ ]:
# Set PyTorch memory allocator to avoid fragmentation (helps with OOM)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
print(" Set PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True")

##  Checkpoint Manager - Resume Training After Runtime Changes

**Tính năng:**
-  Tự động phát hiện checkpoint mới nhất
-  Lưu training state để resume
-  Bảo vệ dữ liệu khi đổi runtime
-  Tracking tiến trình training

**Cách sử dụng:**
1. **Lần đầu train**: Chỉ cần run cell training bình thường
2. **Resume sau khi đổi runtime**: Run cell bên dưới để load checkpoint

In [ ]:
import json
from pathlib import Path
from datetime import datetime

class CheckpointManager:
    """Quản lý checkpoint và training state cho việc resume training"""
    
    def __init__(self, output_dir="./model/outputs/unified"):
        self.output_dir = Path(output_dir)
        self.state_file = self.output_dir / "training_state.json"
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def find_latest_checkpoint(self):
        """Tìm checkpoint mới nhất"""
        checkpoints = sorted(
            [d for d in self.output_dir.glob("checkpoint-*") if d.is_dir()],
            key=lambda x: int(x.name.split("-")[-1])
        )
        return str(checkpoints[-1]) if checkpoints else None
    
    def list_all_checkpoints(self):
        """Liệt kê tất cả checkpoints"""
        checkpoints = sorted(
            [d for d in self.output_dir.glob("checkpoint-*") if d.is_dir()],
            key=lambda x: int(x.name.split("-")[-1])
        )
        return [{"path": str(cp), "step": int(cp.name.split("-")[-1])} for cp in checkpoints]
    
    def save_training_state(self, **kwargs):
        """Lưu thông tin training state"""
        state = {
            "last_update": datetime.now().isoformat(),
            **kwargs
        }
        with open(self.state_file, 'w') as f:
            json.dump(state, f, indent=2)
        print(f" Đã lưu training state: {self.state_file}")
    
    def load_training_state(self):
        """Load training state"""
        if self.state_file.exists():
            with open(self.state_file, 'r') as f:
                return json.load(f)
        return None
    
    def get_resume_info(self):
        """Lấy thông tin để resume training"""
        latest_checkpoint = self.find_latest_checkpoint()
        state = self.load_training_state()
        
        info = {
            "latest_checkpoint": latest_checkpoint,
            "has_checkpoint": latest_checkpoint is not None,
            "training_state": state,
            "all_checkpoints": self.list_all_checkpoints()
        }
        return info
    
    def print_status(self):
        """In ra trạng thái checkpoint"""
        info = self.get_resume_info()
        
        print("\n" + "="*70)
        print(" CHECKPOINT STATUS")
        print("="*70)
        
        if info["has_checkpoint"]:
            print(f" Tìm thấy {len(info['all_checkpoints'])} checkpoint(s)")
            print(f"\n Checkpoint mới nhất: {info['latest_checkpoint']}")
            
            if info["training_state"]:
                print(f"\n Training State:")
                for key, value in info["training_state"].items():
                    print(f"   • {key}: {value}")
            
            print(f"\n Để resume training:")
            print(f"   resume_from_checkpoint='{info['latest_checkpoint']}'")
            print(f"   hoặc")
            print(f"   resume_from_checkpoint='auto'")
        else:
            print("  Chưa có checkpoint nào")
            print("   Training sẽ bắt đầu từ đầu")
        
        print("="*70 + "\n")
        
        return info

#  LƯU Ý: CheckpointManager sẽ được khởi tạo SAU KHI cấu hình OUTPUT_DIR
# (Xem cell Configuration bên dưới)
# Đảm bảo nó sử dụng đúng đường dẫn Drive hoặc local

# Kiểm tra class đã được định nghĩa
print(" CheckpointManager class ready")
print("   Sẽ được khởi tạo với OUTPUT_DIR từ configuration")

###  Nơi Lưu Trữ Checkpoint

**Local (khi chạy trên máy/laptop):**
```
./model/outputs/unified/
 checkpoint-100/
 checkpoint-200/
 checkpoint-300/
 training_state.json
 unified_lora_adapter/  (sau khi train xong)
```

**Google Colab (khi mount Drive):**
```
/content/drive/MyDrive/LexiLingo/models/unified/
 checkpoint-100/
 checkpoint-200/
 checkpoint-300/
 training_state.json
 unified_lora_adapter/
```

**Files quan trọng:**
- `checkpoint-{step}/` - Model weights, optimizer state, scheduler state để resume
- `training_state.json` - Metadata: epoch, step, best loss, learning rate
- `unified_lora_adapter/` - Final trained adapter (sau khi train xong)

##  Bảo Vệ Checkpoint Khi Colab Ngắt Đột Ngột

**Các tình huống ngắt phổ biến:**
-  Hết quota GPU (12h runtime limit)
-  Idle timeout (90 phút không hoạt động)
-  Disconnect mạng
-  Out of memory (OOM)
-  Crash do lỗi

**Giải pháp tự động lưu:**
1.  **Auto-save mỗi 100 steps** (đã có)
2.  **Graceful shutdown handler** (cell bên dưới)
3.  **Save to Google Drive** (persistent storage)
4.  **Keyboard interrupt handler** (Ctrl+C)

In [ ]:
import signal
import sys
import atexit
from datetime import datetime

class GracefulShutdownHandler:
    """
    Handler để tự động lưu checkpoint khi training bị ngắt đột ngột.
    
    Bắt các signal:
    - SIGINT: Ctrl+C (keyboard interrupt)
    - SIGTERM: System shutdown
    - atexit: Python process exit
    """
    
    def __init__(self):
        self.trainer = None
        self.model = None
        self.checkpoint_mgr = None
        self.emergency_save_path = None
        
        # Register signal handlers
        signal.signal(signal.SIGINT, self._signal_handler)
        signal.signal(signal.SIGTERM, self._signal_handler)
        atexit.register(self._emergency_save)
        
        print("  Graceful Shutdown Handler activated")
        print("   • SIGINT (Ctrl+C): ")
        print("   • SIGTERM (shutdown): ")
        print("   • atexit (emergency): \n")
    
    def register_trainer(self, trainer, model, checkpoint_mgr):
        """Đăng ký trainer để có thể save khi cần"""
        self.trainer = trainer
        self.model = model
        self.checkpoint_mgr = checkpoint_mgr
        self.emergency_save_path = Path(trainer.args.output_dir) / "emergency_checkpoint"
        print(f" Trainer registered for auto-save")
        print(f"   Emergency path: {self.emergency_save_path}\n")
    
    def _signal_handler(self, signum, frame):
        """Xử lý khi nhận được signal ngắt"""
        signal_name = "SIGINT" if signum == signal.SIGINT else "SIGTERM"
        print(f"\n\n{'='*70}")
        print(f"  RECEIVED {signal_name} - Training interrupted!")
        print(f"{'='*70}\n")
        
        if self.trainer is not None and self.model is not None:
            try:
                print(" Emergency save in progress...")
                
                # Save checkpoint
                self.emergency_save_path.mkdir(parents=True, exist_ok=True)
                self.model.save_pretrained(str(self.emergency_save_path))
                
                # Save training state
                if self.checkpoint_mgr:
                    self.checkpoint_mgr.save_training_state(
                        status="interrupted",
                        signal=signal_name,
                        timestamp=datetime.now().isoformat(),
                        note=f"Training interrupted by {signal_name}",
                    )
                
                print(f" Emergency checkpoint saved to: {self.emergency_save_path}")
                print(f"   You can resume from this checkpoint later.\n")
                
            except Exception as e:
                print(f" Emergency save failed: {e}")
        else:
            print("  No trainer registered, cannot save checkpoint")
        
        print(f"{'='*70}\n")
        sys.exit(0)
    
    def _emergency_save(self):
        """Emergency save khi Python process exit"""
        # Only save if trainer exists and hasn't been saved yet
        if self.trainer is not None and self.model is not None:
            if not self.emergency_save_path or not self.emergency_save_path.exists():
                print("\n Emergency exit detected - attempting final save...")
                try:
                    self.emergency_save_path.mkdir(parents=True, exist_ok=True)
                    self.model.save_pretrained(str(self.emergency_save_path))
                    print(f" Final checkpoint saved to: {self.emergency_save_path}")
                except:
                    pass  # Silent fail in atexit

# Khởi tạo handler (run ngay khi load notebook)
shutdown_handler = GracefulShutdownHandler()

##  Google Drive Mount - Bảo Vệ Checkpoint Vĩnh Viễn

** QUAN TRỌNG cho Colab:**
Khi hết GPU/timeout, **tất cả dữ liệu local sẽ bị xóa**. Để bảo vệ checkpoint:

###  Bước 1: Mount Google Drive (CHẠY CELL NÀY TRƯỚC)
```python
from google.colab import drive
drive.mount('/content/drive')
```

###  Bước 2: Checkpoint tự động lưu vào Drive
Khi Drive được mount, checkpoint sẽ lưu vào:
```
/content/drive/MyDrive/LexiLingo/models/unified/
```

###  Khi Resume Sau Khi Hết GPU:
1. Mở lại notebook
2. Mount Drive lại (cell trên)
3. Run setup cells (imports, config)
4. Run training cell → **Tự động resume từ checkpoint trên Drive!**

###  Kiểm tra Drive đã mount chưa:
```python
import os
if os.path.exists('/content/drive/MyDrive'):
    print(" Drive mounted")
else:
    print(" Drive chưa mount - checkpoint sẽ mất khi session end!")
```

In [ ]:
#  Kiểm tra Google Drive status (chỉ dành cho Colab)
import os
from pathlib import Path

def check_drive_status():
    """Kiểm tra xem Drive đã được mount chưa"""
    drive_path = Path("/content/drive/MyDrive")
    
    print("\n" + "="*70)
    print(" GOOGLE DRIVE STATUS")
    print("="*70)
    
    if drive_path.exists():
        print(" Google Drive đã mount")
        print(f"   Path: {drive_path}")
        
        # Kiểm tra LexiLingo folder
        lexilingo_path = drive_path / "LexiLingo"
        if lexilingo_path.exists():
            print(f"\n LexiLingo folder exists")
            print(f"   Path: {lexilingo_path}")
            
            # List contents
            contents = list(lexilingo_path.iterdir())
            print(f"   Contents: {len(contents)} items")
            for item in contents:
                print(f"   - {item.name}")
        
        # Kiểm tra checkpoint folder mới
        checkpoint_path = drive_path / "LexiLingo/unified_model"
        if checkpoint_path.exists():
            checkpoints = list(checkpoint_path.glob("checkpoint-*"))
            print(f"\n Checkpoint folder exists: unified_model/")
            print(f"   {len(checkpoints)} checkpoint(s) found")
        else:
            print(f"\n Checkpoint folder sẽ được tạo tự động khi training")
            print(f"   Path: {checkpoint_path}")
        
        print(f"\n Checkpoint sẽ được lưu vào Drive (persistent)")
        print(f"    An toàn khi Colab disconnect/timeout")
        
    else:
        print("  Google Drive CHƯA mount")
        print(f"\n CẢNH BÁO:")
        print(f"   • Checkpoint sẽ lưu vào /content/ (local)")
        print(f"   • SẼ MẤT TẤT CẢ khi session end!")
        print(f"\n Giải pháp:")
        print(f"   1. Run cell mount Drive ở trên")
        print(f"   2. Hoặc chạy:")
        print(f"      from google.colab import drive")
        print(f"      drive.mount('/content/drive')")
    
    print("="*70 + "\n")

# Chỉ check khi chạy trên Colab
try:
    import google.colab
    check_drive_status()
except ImportError:
    print("ℹ  Đang chạy local (không phải Colab)")
    print(f"   Checkpoint sẽ lưu vào: ./model/outputs/unified/\n")

## 2. Configuration

In [ ]:
#  Kiểm tra checkpoint trước khi config
# Cell này sẽ hiển thị có checkpoint nào để resume không
print("\n Checking for existing checkpoints before configuration...\n")

# Tạo output directory nếu chưa có
import os
from pathlib import Path

DRIVE_OUT = "/content/drive/MyDrive/LexiLingo/models"
LOCAL_OUT = "./model/outputs"  # Lưu vào folder model/outputs trong workspace
BASE_OUT = DRIVE_OUT if Path(DRIVE_OUT).exists() else LOCAL_OUT
OUTPUT_DIR_TEMP = str(Path(BASE_OUT) / "unified")
Path(OUTPUT_DIR_TEMP).mkdir(parents=True, exist_ok=True)

print(f" Output directory: {OUTPUT_DIR_TEMP}")
print(f"   (Trong workspace: {Path(OUTPUT_DIR_TEMP).resolve()})\n")

# Khởi tạo checkpoint manager tạm để check
checkpoint_mgr_temp = CheckpointManager(OUTPUT_DIR_TEMP)
resume_info_temp = checkpoint_mgr_temp.get_resume_info()

if resume_info_temp["has_checkpoint"]:
    print(f"Found existing checkpoint: {resume_info_temp['latest_checkpoint']}")
    print(f"Training sẽ tự động resume từ checkpoint này")
else:
    print("No existing checkpoint found")

    print("Training sẽ bắt đầu từ đầu")
print("\n" + "="*70)


In [4]:
# Model configuration (architecture.md: Unified Adapter)
# OPTIMIZED: Sử dụng model nhỏ hơn và giảm sequence length
MODEL_NAME = "Qwen/Qwen2.5-0.5B-Instruct"  # Giảm từ 1.5B xuống 0.5B (nhanh hơn 3x)
MAX_SEQ_LENGTH = 512  # Giảm từ 768 xuống 512 (giảm ~33% memory + faster)

# UNIFIED LoRA Configuration - OPTIMIZED
# Giảm LoRA rank để có ít parameters hơn và training nhanh hơn
UNIFIED_LORA_CONFIG = {
    "task_type": TaskType.CAUSAL_LM,
    "r": 16,  # Giảm từ 48 xuống 16 (giảm ~66% LoRA params, nhanh hơn nhiều)
    "lora_alpha": 32,  # Giảm từ 96 xuống 32 (tỷ lệ với r)
    "lora_dropout": 0.1,  # Tăng từ 0.05 lên 0.1 (regularization tốt hơn với rank thấp)
    "bias": "none",
    "target_modules": [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    "inference_mode": False,
}

# Output paths - BẮT BUỘC lưu vào Google Drive
DRIVE_MOUNT = "/content/drive"
DRIVE_BASE = "/content/drive/MyDrive/LexiLingo"

# KIỂM TRA Drive đã mount chưa
if not Path(DRIVE_MOUNT).exists():
    raise RuntimeError(
        "\nERROR: Google Drive chưa được mount!\n"
        "Vui lòng:\n"
        "1. Chạy cell mount Drive ở trên\n"
        "2. Cho phép quyền truy cập Drive\n"
        "3. Chạy lại cell này\n"
    )

print("\n" + "="*70)
print("GOOGLE DRIVE STATUS: MOUNTED")
print("="*70)

# Tạo LexiLingo folder nếu chưa có
Path(DRIVE_BASE).mkdir(parents=True, exist_ok=True)
print(f"LexiLingo folder: {DRIVE_BASE}")

# Lưu checkpoints vào unified_model_optimized (folder mới để phân biệt)
OUTPUT_DIR = str(Path(DRIVE_BASE) / "unified_model_optimized")

# Tạo output directory
Path(OUTPUT_DIR).mkdir(parents=True, exist_ok=True)

# Test ghi file để đảm bảo có quyền write vào Drive
test_file = Path(OUTPUT_DIR) / ".write_test"
try:
    test_file.write_text("test")
    test_file.unlink()
    print(f"Checkpoint directory: {OUTPUT_DIR}")
    print("Write permission: OK")
except Exception as e:
    raise RuntimeError(f"Không thể ghi vào Drive! Error: {e}")

if Path(OUTPUT_DIR).exists():
    print("Directory verification: OK")
else:
    print("WARNING: Failed to create directory!")
    
print("="*70)
print(f"⚡ OPTIMIZED CONFIG:")
print(f"  - Model: Qwen2.5-0.5B (3x faster than 1.5B)")
print(f"  - Sequence length: 512 (33% less memory)")
print(f"  - LoRA rank: 16 (66% fewer params)")
print(f"Checkpoints sẽ lưu vào Drive tại:")
print(f"  MyDrive/LexiLingo/unified_model_optimized/")
print("="*70 + "\n")

# Training configuration - OPTIMIZED
if torch.cuda.is_available():
    # Optimized for T4 GPU: Faster training với model nhỏ hơn
    TRAINING_CONFIG = {
        "output_dir": OUTPUT_DIR,
        "num_train_epochs": 4,  # Giảm từ 7 xuống 4 epochs (đủ với model nhỏ)
        "per_device_train_batch_size": 4,  # Tăng từ 2 lên 4 (model nhỏ hơn, fit được)
        "per_device_eval_batch_size": 4,
        "gradient_accumulation_steps": 6,  # Giảm từ 12 xuống 6 (effective batch vẫn = 24)
        "learning_rate": 3e-4,  # Tăng từ 2e-4 (model nhỏ có thể train với LR cao hơn)
        "weight_decay": 0.01,
        "warmup_ratio": 0.05,  # Tăng warmup ratio (ổn định hơn với LR cao)
        "lr_scheduler_type": "cosine",
        "logging_steps": 10,
        "save_steps": 150,  # Giảm tần suất save (từ 100->150, ít I/O hơn)
        "eval_steps": 150,
        "save_total_limit": 2,  # Giảm từ 3 xuống 2 (tiết kiệm disk space)
        "save_strategy": "steps",
        "load_best_model_at_end": True,
        "fp16": bool(use_fp16),
        "bf16": bool(use_bf16),
        "gradient_checkpointing": True,  # Vẫn giữ để save memory
        "optim": "paged_adamw_8bit",  # Đổi từ 32bit sang 8bit (nhanh hơn, ít memory hơn)
        "report_to": "none",
        "dataloader_num_workers": 2,
        "max_grad_norm": 1.0,
    }
else:
    # CPU/MPS fallback
    TRAINING_CONFIG = {
        "output_dir": OUTPUT_DIR,
        "num_train_epochs": 2,  # Giảm từ 3 xuống 2 cho CPU
        "per_device_train_batch_size": 2,
        "per_device_eval_batch_size": 2,
        "gradient_accumulation_steps": 12,
        "learning_rate": 3e-4,
        "weight_decay": 0.01,
        "warmup_ratio": 0.05,
        "lr_scheduler_type": "cosine",
        "logging_steps": 10,
        "save_steps": 100,
        "eval_steps": 100,
        "save_total_limit": 2,
        "save_strategy": "steps",
        "load_best_model_at_end": True,
        "fp16": False,
        "bf16": False,
        "gradient_checkpointing": True,
        "optim": "adamw_torch",
        "report_to": "none",
        "dataloader_num_workers": 2,
        "max_grad_norm": 1.0,
    }

# Dataset target sizes (architecture.md)
DATASET_TARGETS = {
    "fluency": 1500,
    "grammar": 9200,
    "vocabulary": 2500,
    "dialogue": 5200,
}

Path(TRAINING_CONFIG["output_dir"]).mkdir(parents=True, exist_ok=True)

# QUAN TRỌNG: Khởi tạo CheckpointManager với OUTPUT_DIR đã được config
checkpoint_mgr = CheckpointManager(TRAINING_CONFIG["output_dir"])

# Verify storage location
print("\n" + "="*70)
print("⚡ OPTIMIZED CONFIGURATION SUMMARY")
print("="*70)
print(f"Model: {MODEL_NAME}")
print(f"  └─ Size: 0.5B params (3x faster than 1.5B)")
print(f"  └─ Memory: ~2GB (vs ~6GB for 1.5B)")
print(f"\nSequence & LoRA:")
print(f"  └─ MAX_SEQ_LENGTH: {MAX_SEQ_LENGTH} (vs 768 before)")
print(f"  └─ LoRA rank: {UNIFIED_LORA_CONFIG['r']} (vs 48 before)")
print(f"  └─ LoRA alpha: {UNIFIED_LORA_CONFIG['lora_alpha']} (vs 96 before)")
print(f"  └─ LoRA dropout: {UNIFIED_LORA_CONFIG['lora_dropout']}")
print(f"\nTraining Speed Improvements:")
print(f"  ✓ Batch size: {TRAINING_CONFIG['per_device_train_batch_size']} (2x larger)")
print(f"  ✓ Gradient accumulation: {TRAINING_CONFIG['gradient_accumulation_steps']} (2x less)")
print(f"  ✓ Epochs: {TRAINING_CONFIG['num_train_epochs']} (vs 7 before)")
print(f"  ✓ Optimizer: paged_adamw_8bit (vs 32bit)")
print(f"\nEstimated training time reduction: 60-70% faster")
print(f"\nOutput directory: {TRAINING_CONFIG['output_dir']}")
print(f"Checkpoints saved every {TRAINING_CONFIG['save_steps']} steps")
print(f"Max checkpoints kept: {TRAINING_CONFIG['save_total_limit']}")
print(f"Precision: fp16={TRAINING_CONFIG['fp16']} bf16={TRAINING_CONFIG['bf16']}")

# Verify write permissions
try:
    test_file = Path(TRAINING_CONFIG['output_dir']) / '.write_test'
    test_file.write_text('test')
    test_file.unlink()
    print("\n✓ Write permission: OK")
except Exception as e:
    print(f"\n⚠ WARNING: Cannot write to output directory: {e}")
    
print("="*70 + "\n")

## ⚡ Tối Ưu Hóa Training - So Sánh

### Các thay đổi để training nhanh hơn 60-70%:

| Thông số | Trước | Sau (Optimized) | Cải thiện |
|----------|-------|-----------------|-----------|
| **Model** | Qwen2.5-1.5B | Qwen2.5-0.5B | **3x nhanh hơn** |
| **Sequence Length** | 768 | 512 | **33% ít memory** |
| **LoRA rank (r)** | 48 | 16 | **66% ít params** |
| **LoRA alpha** | 96 | 32 | Tỷ lệ với rank |
| **Dropout** | 0.05 | 0.1 | Regularization tốt hơn |
| **Epochs** | 7 | 4 | **43% ít epochs** |
| **Batch size** | 2 | 4 | **2x lớn hơn** |
| **Gradient accumulation** | 12 | 6 | **2x ít steps** |
| **Learning rate** | 2e-4 | 3e-4 | Train nhanh hơn |
| **Optimizer** | adamw_32bit | adamw_8bit | Ít memory hơn |
| **Save steps** | 100 | 150 | Ít I/O hơn |

### Tại sao nhanh hơn?

1. **Model nhỏ hơn (0.5B vs 1.5B)**: 
   - Forward pass nhanh gấp 3x
   - Memory footprint thấp hơn (~2GB vs ~6GB)
   - Cho phép batch size lớn hơn

2. **LoRA rank thấp hơn (16 vs 48)**:
   - Ít trainable parameters hơn 66%
   - Backward pass nhanh hơn đáng kể
   - Vẫn đủ capacity cho task này

3. **Sequence length ngắn hơn (512 vs 768)**:
   - Attention mechanism nhanh hơn (O(n²) complexity)
   - Giảm 33% memory usage
   - Phù hợp với hầu hết inputs

4. **Ít epochs hơn (4 vs 7)**:
   - Model nhỏ converge nhanh hơn
   - Giảm 43% total training time

5. **Batch size lớn hơn + ít gradient accumulation**:
   - Throughput cao hơn (4 samples/step vs 2)
   - Ít backward passes hơn (6 vs 12)
   - Tận dụng tốt GPU parallelism

### Trade-offs:

- **Chất lượng**: Model 0.5B có thể kém 1.5B ~5-10% accuracy, nhưng vẫn rất tốt cho production
- **Capacity**: LoRA rank 16 đủ cho fine-tuning nhưng ít expressive hơn rank 48
- **Generalization**: Cần monitor validation loss để tránh underfitting

### Khi nào nên dùng config này?

✅ **NÊN dùng khi**:
- Cần iterate nhanh (prototyping, testing)
- Resource hạn chế (GPU nhỏ, thời gian ít)
- Task không quá phức tạp
- Ưu tiên inference speed

❌ **KHÔNG nên dùng khi**:
- Cần accuracy tối đa
- Có đủ GPU power và thời gian
- Task cực kỳ phức tạp

In [ ]:
# 📊 Training Speed Estimator - So sánh thời gian training

def estimate_training_time(config_name, model_size, seq_len, lora_r, batch_size, 
                          grad_accum, epochs, dataset_size=18400):
    """
    Ước tính thời gian training dựa trên hardware benchmarks
    
    Args:
        config_name: Tên config
        model_size: Số parameters (B)
        seq_len: Sequence length
        lora_r: LoRA rank
        batch_size: Per device batch size
        grad_accum: Gradient accumulation steps
        epochs: Số epochs
        dataset_size: Tổng số samples
    """
    
    # Base time per sample trên T4 GPU (milliseconds)
    # Qwen 0.5B: ~80ms, Qwen 1.5B: ~240ms
    base_time_per_sample = {
        0.5: 80,   # ms
        1.5: 240,  # ms
    }
    
    # Adjustments
    time_ms = base_time_per_sample[model_size]
    
    # Sequence length adjustment (quadratic for attention)
    seq_factor = (seq_len / 512) ** 1.5
    time_ms *= seq_factor
    
    # LoRA rank adjustment (more params = slower backward)
    lora_factor = 1 + (lora_r / 100)  # r=16 -> 1.16x, r=48 -> 1.48x
    time_ms *= lora_factor
    
    # Batch size efficiency (larger batch = better GPU utilization)
    batch_efficiency = min(1.0, 0.5 + (batch_size / 8))
    time_ms *= (2 - batch_efficiency)  # Inverse: larger batch = faster per sample
    
    # Calculate total
    effective_batch = batch_size * grad_accum
    steps_per_epoch = dataset_size / effective_batch
    total_steps = steps_per_epoch * epochs
    
    # Time per step = time_ms * batch_size (forward + backward + optimizer)
    time_per_step_sec = (time_ms * batch_size * grad_accum) / 1000
    
    total_time_sec = time_per_step_sec * total_steps
    hours = total_time_sec / 3600
    
    print(f"\n{'='*70}")
    print(f"  {config_name}")
    print(f"{'='*70}")
    print(f"Model: Qwen2.5-{model_size}B")
    print(f"  ├─ Sequence length: {seq_len}")
    print(f"  ├─ LoRA rank: {lora_r}")
    print(f"  ├─ Batch size: {batch_size} × {grad_accum} = {effective_batch}")
    print(f"  └─ Epochs: {epochs}")
    print(f"\nDataset: {dataset_size:,} samples")
    print(f"  ├─ Steps per epoch: {steps_per_epoch:.0f}")
    print(f"  └─ Total steps: {total_steps:.0f}")
    print(f"\nEstimated Time:")
    print(f"  ├─ Per step: {time_per_step_sec:.2f}s")
    print(f"  ├─ Per epoch: {(time_per_step_sec * steps_per_epoch / 60):.1f} min")
    print(f"  └─ Total: {hours:.2f} hours ({total_time_sec/60:.0f} minutes)")
    print(f"{'='*70}\n")
    
    return total_time_sec

# Compare configs
print("\n🚀 TRAINING TIME COMPARISON (on T4 GPU)\n")

# Old config
old_time = estimate_training_time(
    config_name="❌ OLD CONFIG (Slow)",
    model_size=1.5,
    seq_len=768,
    lora_r=48,
    batch_size=2,
    grad_accum=12,
    epochs=7
)

# New optimized config
new_time = estimate_training_time(
    config_name="✅ NEW OPTIMIZED CONFIG (Fast)",
    model_size=0.5,
    seq_len=512,
    lora_r=16,
    batch_size=4,
    grad_accum=6,
    epochs=4
)

# Summary
speedup = old_time / new_time
time_saved = old_time - new_time

print("\n" + "="*70)
print("  📊 PERFORMANCE IMPROVEMENT SUMMARY")
print("="*70)
print(f"Old config total time: {old_time/3600:.2f} hours")
print(f"New config total time: {new_time/3600:.2f} hours")
print(f"\n🎯 Speedup: {speedup:.2f}x faster")
print(f"⏰ Time saved: {time_saved/3600:.2f} hours ({time_saved/60:.0f} minutes)")
print(f"📉 Reduction: {((old_time - new_time) / old_time * 100):.1f}%")
print("="*70)

# Memory estimate
print("\n" + "="*70)
print("  💾 MEMORY USAGE ESTIMATE (GPU)")
print("="*70)
print("Old config (1.5B):")
print("  ├─ Model: ~6 GB")
print("  ├─ Activations (seq=768, bs=2): ~3 GB")
print("  ├─ Optimizer states: ~2 GB")
print("  └─ Total: ~11 GB (tight on T4)")
print("\nNew config (0.5B):")
print("  ├─ Model: ~2 GB")
print("  ├─ Activations (seq=512, bs=4): ~2 GB")
print("  ├─ Optimizer states: ~1 GB")
print("  └─ Total: ~5 GB (comfortable on T4)")
print("\n💡 Memory reduction: ~55% (11GB → 5GB)")
print("="*70 + "\n")

## 3. Load Base Model & Tokenizer

In [5]:
# Load tokenizer + base model (Colab GPU recommended)
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_NAME,
    padding_side="right",
 )

print(f"\nTokenizer loaded:")
print(f"  Vocab size: {tokenizer.vocab_size}")

if torch.cuda.is_available():
    print("\nLoading model with 4-bit quantization (bitsandbytes)...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16 if use_bf16 else torch.float16,
        bnb_4bit_use_double_quant=True,
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        low_cpu_mem_usage=True,
    )
else:
    # CPU/MPS fallback: load full precision (no bnb 4-bit on CPU)
    print("\n No CUDA detected. Loading model without 4-bit quantization...")
    base_model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        device_map="auto",
        low_cpu_mem_usage=True,
    )

print(f"\nModel loaded:")
print(f"  Model vocab size: {base_model.config.vocab_size}")
print(f"  Embedding size: {base_model.get_input_embeddings().weight.shape[0]}")

# FIX: Handle vocab size mismatch (prevents CUDA device-side assert)
if base_model.config.vocab_size != tokenizer.vocab_size:
    print(f"\n⚠️  Vocab size mismatch detected!")
    print(f"  Model: {base_model.config.vocab_size}")
    print(f"  Tokenizer: {tokenizer.vocab_size}")
    print(f"  Difference: {abs(base_model.config.vocab_size - tokenizer.vocab_size)}")
    
    # Resize model embeddings to match tokenizer
    print(f"\n  → Resizing model embeddings to {tokenizer.vocab_size}...")
    base_model.resize_token_embeddings(tokenizer.vocab_size)
    base_model.config.vocab_size = tokenizer.vocab_size
    
    print(f"  ✓ Model resized to match tokenizer")
    print(f"  New embedding size: {base_model.get_input_embeddings().weight.shape[0]}")

# Set pad token AFTER resizing (ensures token ID is within valid range)
tokenizer.pad_token = tokenizer.eos_token

# Validate special tokens
print(f"\nSpecial token validation:")
print(f"  pad_token_id: {tokenizer.pad_token_id} (valid: {tokenizer.pad_token_id < tokenizer.vocab_size})")
print(f"  eos_token_id: {tokenizer.eos_token_id} (valid: {tokenizer.eos_token_id < tokenizer.vocab_size})")

# Enable training-friendly settings
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

print(f"\n✓ Model setup complete: {MODEL_NAME}")

## 4. Prepare Training Data

### 4.1 Fluency Scoring Dataset

In [6]:
# ============================================================================
# Load cleaned + anti-leakage split data from Drive (recommended)
# Expects: train.jsonl / val.jsonl in downloaded_datasets/
# ============================================================================
from pathlib import Path
import json
from datasets import load_dataset, Dataset

DRIVE_DATA_PATH = "/content/drive/MyDrive/LexiLingo/training_data/downloaded_datasets"
LOCAL_DATA_PATH = "./downloaded_datasets"

# Resolve data directory
if Path(DRIVE_DATA_PATH).exists():
    data_dir = Path(DRIVE_DATA_PATH)
    print(f" Found data in Google Drive: {data_dir}")
elif Path(LOCAL_DATA_PATH).exists():
    data_dir = Path(LOCAL_DATA_PATH)
    print(f" Found data locally: {data_dir}")
else:
    raise FileNotFoundError(
        "No dataset folder found. Put downloaded_datasets/ into Drive at: "
        f"{DRIVE_DATA_PATH}"
    )

# Optional: show split report (leakage-safe split)
report_path = data_dir / "split_report.json"
if report_path.exists():
    rep = json.loads(report_path.read_text(encoding="utf-8"))
    leakage = rep.get("split", {}).get("leakage_groups")
    train_n = rep.get("split", {}).get("train_samples")
    val_n = rep.get("split", {}).get("val_samples")
    print("\n split_report.json")
    print(f"  leakage_groups: {leakage}")
    print(f"  train_samples:  {train_n}")
    print(f"  val_samples:    {val_n}")

train_jsonl = data_dir / "train.jsonl"
val_jsonl = data_dir / "val.jsonl"
unified_json = data_dir / "unified_training_data.json"

if train_jsonl.exists() and val_jsonl.exists():
    print("\n Loading JSONL split (anti-leakage)...")
    train_raw = load_dataset("json", data_files=str(train_jsonl), split="train")
    val_raw = load_dataset("json", data_files=str(val_jsonl), split="train")
    print(f"  Train: {len(train_raw)}")
    print(f"  Val:   {len(val_raw)}")
elif unified_json.exists():
    print("\n train/val JSONL not found. Falling back to unified_training_data.json")
    with open(unified_json, "r", encoding="utf-8") as f:
        unified_training_data = json.load(f)
    raw = Dataset.from_list(unified_training_data)
    split = raw.train_test_split(test_size=0.05, seed=42)
    train_raw = split["train"]
    val_raw = split["test"]
    print(f"  Train: {len(train_raw)}")
    print(f"  Val:   {len(val_raw)}")
else:
    raise FileNotFoundError("Missing train.jsonl/val.jsonl or unified_training_data.json")

# Task mapping (architecture.md naming)
TASK_NAME_MAP = {
    "fluency": "fluency_scoring",
    "vocabulary": "vocabulary_classification",
    "grammar": "grammar_correction",
    "dialogue": "dialogue_response",
}

SYSTEM_PROMPT = (
    "You are LexiLingo's unified English tutor model. "
    "Follow the task instruction and respond ONLY with valid JSON (no extra text)."
 )

def _safe_get(dct, *keys, default=None):
    cur = dct
    for k in keys:
        if not isinstance(cur, dict) or k not in cur:
            return default
        cur = cur[k]
    return cur

def format_unified_prompt(example):
    """Format 1 record into Qwen chat template (unified multi-task)."""
    task = example.get("task")
    task_name = TASK_NAME_MAP.get(task, task or "unknown")
    user_text = example.get("input", "")
    output_obj = example.get("output", {})
    metadata = example.get("metadata", {}) if isinstance(example.get("metadata"), dict) else {}

    if task_name in ("fluency_scoring", "vocabulary_classification", "grammar_correction"):
        prompt = f"Task: {task_name}\nText: {user_text}"
    else:
        history = metadata.get("history") or metadata.get("conversation_history") or ""
        strategy = _safe_get(output_obj, "strategy") or metadata.get("strategy") or "socratic_questioning"
        prompt = (
            f"Task: {task_name}\n"
            f"Text: {user_text}\n"
            f"Context: {history}\n"
            f"Strategy: {strategy}"
        )

    # Ensure assistant content is JSON string
    if isinstance(output_obj, (dict, list)):
        response = json.dumps(output_obj, ensure_ascii=False)
    else:
        response = json.dumps({"response": str(output_obj)}, ensure_ascii=False)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": prompt},
        {"role": "assistant", "content": response},
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
    return {"text": text, "task": task}

print("\n Formatting train/val with chat templates...")
train_dataset = train_raw.map(format_unified_prompt)
val_dataset = val_raw.map(format_unified_prompt)

print(f" Train ready: {len(train_dataset)}")
print(f" Val ready:   {len(val_dataset)}")
print("\n Example formatted prompt:")
print("=" * 60)
print(train_dataset[0]["text"][:600] + "...")
print("=" * 60)

## 5. MongoDB Logging Middleware (Optional)

In [18]:
# MongoDB Logging Middleware (based on architecture.md)
import pymongo
from datetime import datetime
from typing import Dict, Any

# Safety: if cells executed out-of-order, ensure MONGODB_CONFIG exists
if "MONGODB_CONFIG" not in globals():
    MONGODB_CONFIG = {
        "enabled": False,
        "connection_string": "mongodb://localhost:27017/",
        "database": "lexilingo_training",
        "collections": {
            "training_logs": "training_logs",
            "model_metrics": "model_metrics",
            "training_queue": "training_queue",
        },
    }
    print(" MONGODB_CONFIG was not defined yet → using default (disabled)")

class MongoDBLogger:
    """
    Logging middleware for training metrics
    Stores training logs in MongoDB for analysis
    """
    def __init__(self, config: Dict[str, Any]):
        self.enabled = bool(config.get("enabled", False))
        if not self.enabled:
            print("  MongoDB logging disabled")
            return
        
        try:
            self.client = pymongo.MongoClient(
                config["connection_string"],
                serverSelectionTimeoutMS=5000,
            )
            self.db = self.client[config["database"]]
            
            # Collections
            self.training_logs = self.db[config["collections"]["training_logs"]]
            self.model_metrics = self.db[config["collections"]["model_metrics"]]
            self.training_queue = self.db[config["collections"]["training_queue"]]
            
            # Test connection
            self.client.server_info()
            print(" MongoDB connected successfully")
            
            # Create indexes
            self.training_logs.create_index([("timestamp", -1)])
            self.model_metrics.create_index([("epoch", 1)])
            
        except Exception as e:
            print(f"  MongoDB connection failed: {e}")
            self.enabled = False
    
    def log_training_step(self, step: int, loss: float, learning_rate: float, task: str = None):
        """Log training step metrics"""
        if not self.enabled:
            return
        
        try:
            self.training_logs.insert_one({
                "timestamp": datetime.now(),
                "step": step,
                "loss": float(loss),
                "learning_rate": float(learning_rate),
                "task": task,
                "model": "qwen2.5-1.5b-unified",
            })
        except Exception as e:
            print(f"  Failed to log step: {e}")
    
    def log_epoch_metrics(self, epoch: int, metrics: Dict[str, float]):
        """Log epoch-level metrics"""
        if not self.enabled:
            return
        
        try:
            payload = {k: (float(v) if isinstance(v, (int, float)) else v) for k, v in metrics.items()}
            self.model_metrics.insert_one({
                "timestamp": datetime.now(),
                "epoch": int(epoch),
                "model": "qwen2.5-1.5b-unified",
                **payload,
            })
        except Exception as e:
            print(f"  Failed to log epoch: {e}")
    
    def close(self):
        """Close MongoDB connection"""
        if getattr(self, "enabled", False):
            self.client.close()

# Initialize logger (won't crash if config missing)
mongo_logger = MongoDBLogger(MONGODB_CONFIG)

print("\n To enable MongoDB logging:")
print("  1) Set MONGODB_CONFIG['enabled'] = True")
print("  2) Provide a reachable MongoDB URI (Atlas or local)")
print("  3) Re-run this cell")

## 6. Fine-tune Unified Adapter

###  Hệ Thống Checkpoint - Tự Động Tiếp Tục Train

**Pipeline này có khả năng tự động tiếp tục train khi bị gián đoạn:**

 **Khi nào checkpoint hoạt động:**
- Colab bị disconnect (hết thời gian, lỗi mạng)
- Hết dung lượng RAM/GPU (crash)
- Bạn tự tắt training (Ctrl+C hoặc stop cell)
- Session timeout

 **Cách hoạt động:**
- **Tự động lưu checkpoint** mỗi 200 steps vào Drive:
  ```
  /content/drive/MyDrive/LexiLingo/models/unified_adapter/
     checkpoint-200/
     checkpoint-400/
     checkpoint-600/
  ```
- Giữ **3 checkpoint gần nhất** (tiết kiệm dung lượng Drive)
- Khi re-run cell training, **tự động phát hiện** checkpoint mới nhất và tiếp tục từ đó

 **Cách sử dụng:**

1. **Tự động resume (mặc định):**
   ```python
   resume_from_checkpoint="auto"  # ← Đã set sẵn
   ```
   → Tự động tìm checkpoint mới nhất và tiếp tục

2. **Resume từ checkpoint cụ thể:**
   ```python
   resume_from_checkpoint="checkpoint-1000"
   ```

3. **Train từ đầu (xóa progress cũ):**
   ```python
   resume_from_checkpoint=None
   ```

 **Lưu ý quan trọng:**
- Checkpoint được lưu vào **Drive** nên an toàn khi Colab disconnect
- **KHÔNG cần** làm gì thêm - chỉ cần re-run cell training sau khi restart runtime
- Xem progress: ` Auto-detected checkpoint: checkpoint-1200` → đang tiếp tục từ step 1200

 **Nếu hết dung lượng Drive:**
- Xóa checkpoint cũ: `!rm -rf /content/drive/MyDrive/LexiLingo/models/unified_adapter/checkpoint-*`
- Giảm `save_total_limit=3` xuống `save_total_limit=2` (chỉ giữ 2 checkpoint)

In [ ]:
def finetune_unified_adapter(train_dataset, eval_dataset, lora_config, resume_from_checkpoint=None):
    """
    Fine-tune base model with UNIFIED LoRA adapter for multi-task learning.

    Args:
        train_dataset: Hugging Face Dataset with column 'text' (already chat-templated)
        eval_dataset: Hugging Face Dataset with column 'text' (already chat-templated)
        lora_config: LoRA configuration dict
        resume_from_checkpoint: Path to checkpoint folder to resume training
                              - "auto": Auto-detect latest checkpoint
                              - "/path/to/checkpoint-1000": Resume from specific checkpoint
                              - None: Start fresh

    Returns:
        (model, trainer)
    """
    print(f"\n{'='*60}")
    print("Training UNIFIED LoRA Adapter (Multi-Task Learning)")
    print(f"{'='*60}\n")
    
    # Auto-detect latest checkpoint if resume_from_checkpoint="auto"
    if resume_from_checkpoint == "auto":
        latest_checkpoint = checkpoint_mgr.find_latest_checkpoint()
        if latest_checkpoint:
            resume_from_checkpoint = latest_checkpoint
            print(f" Auto-detected checkpoint: {resume_from_checkpoint}")
            
            # Load previous training state
            prev_state = checkpoint_mgr.load_training_state()
            if prev_state:
                print(f" Previous training info:")
                print(f"   • Last update: {prev_state.get('last_update', 'N/A')}")
                print(f"   • Epoch: {prev_state.get('epoch', 'N/A')}")
                print(f"   • Global step: {prev_state.get('global_step', 'N/A')}")
                print(f"   • Best eval loss: {prev_state.get('best_eval_loss', 'N/A')}")
        else:
            resume_from_checkpoint = None
            print("ℹ  No checkpoint found, starting from scratch")
    
    if resume_from_checkpoint:
        print(f" Resuming from checkpoint: {resume_from_checkpoint}\n")

    peft_config = LoraConfig(**lora_config)

    # Prepare model for training
    model = base_model
    model = prepare_model_for_kbit_training(model)
    model = get_peft_model(model, peft_config)

    # Print trainable parameters
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    total_params = sum(p.numel() for p in model.parameters())
    print(f" Model info:")
    print(f"   • Trainable params: {trainable_params:,} ({100 * trainable_params / total_params:.2f}%)")
    print(f"   • LoRA rank: {lora_config['r']}")
    print(f"   • LoRA alpha: {lora_config['lora_alpha']}")
    print(f"   • Target modules: {lora_config['target_modules']}")

    # Tokenize datasets (text -> input_ids + labels)
    def tokenize_function(examples):
        result = tokenizer(
            examples["text"],
            truncation=True,
            max_length=MAX_SEQ_LENGTH,
            padding="max_length",
        )
        result["labels"] = result["input_ids"].copy()
        return result

    print("\n Tokenizing datasets...")
    train_tokenized = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=train_dataset.column_names,
        desc="Tokenizing train",
    )
    eval_tokenized = eval_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=eval_dataset.column_names,
        desc="Tokenizing eval",
    )

    # Training arguments with enhanced checkpoint strategy
    training_args = TrainingArguments(
        output_dir=TRAINING_CONFIG['output_dir'],
        num_train_epochs=TRAINING_CONFIG['num_train_epochs'],
        per_device_train_batch_size=TRAINING_CONFIG['per_device_train_batch_size'],
        per_device_eval_batch_size=TRAINING_CONFIG['per_device_eval_batch_size'],
        gradient_accumulation_steps=TRAINING_CONFIG['gradient_accumulation_steps'],
        learning_rate=TRAINING_CONFIG['learning_rate'],
        weight_decay=TRAINING_CONFIG['weight_decay'],
        warmup_ratio=TRAINING_CONFIG['warmup_ratio'],
        lr_scheduler_type=TRAINING_CONFIG['lr_scheduler_type'],
        logging_steps=TRAINING_CONFIG['logging_steps'],
        save_steps=TRAINING_CONFIG['save_steps'],
        eval_steps=TRAINING_CONFIG['eval_steps'],
        save_total_limit=TRAINING_CONFIG['save_total_limit'],
        fp16=TRAINING_CONFIG['fp16'],
        bf16=TRAINING_CONFIG['bf16'],
        gradient_checkpointing=TRAINING_CONFIG['gradient_checkpointing'],
        max_grad_norm=TRAINING_CONFIG['max_grad_norm'],
        optim=TRAINING_CONFIG['optim'],
        report_to=TRAINING_CONFIG['report_to'],
        logging_first_step=True,
        eval_strategy="steps",
        save_strategy="steps",
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,
        dataloader_num_workers=TRAINING_CONFIG['dataloader_num_workers'],
        dataloader_pin_memory=torch.cuda.is_available(),
        # Enhanced checkpoint settings
        save_on_each_node=False,
        save_safetensors=True,
        resume_from_checkpoint=resume_from_checkpoint,
    )

    print(f"\n Dataset:")
    print(f"   • Train: {len(train_tokenized)} samples")
    print(f"   • Eval:  {len(eval_tokenized)} samples")
    
    print(f"\n  Training config:")
    print(f"   • Epochs: {training_args.num_train_epochs}")
    print(f"   • Batch size: {training_args.per_device_train_batch_size}")
    print(f"   • Gradient accumulation: {training_args.gradient_accumulation_steps}")
    print(f"   • Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
    print(f"   • Learning rate: {training_args.learning_rate}")
    print(f"   • Save steps: {training_args.save_steps}")
    print(f"   • Eval steps: {training_args.eval_steps}")
    print(f"   • Save total limit: {training_args.save_total_limit}")

    # Data collator for causal LM
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False,
    )

    # Custom callback to save training state
    from transformers import TrainerCallback
    
    class CheckpointCallback(TrainerCallback):
        """Callback để lưu training state sau mỗi checkpoint"""
        
        def on_save(self, args, state, control, **kwargs):
            """Called after a checkpoint save"""
            checkpoint_mgr.save_training_state(
                epoch=state.epoch,
                global_step=state.global_step,
                best_metric=state.best_metric,
                best_model_checkpoint=state.best_model_checkpoint,
                total_epochs=args.num_train_epochs,
                learning_rate=args.learning_rate,
            )
            print(f" Checkpoint saved at step {state.global_step} (epoch {state.epoch:.2f})")

    # Standard Trainer with callback
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_tokenized,
        eval_dataset=eval_tokenized,
        data_collator=data_collator,
        callbacks=[CheckpointCallback()],
    )

    print("\n Starting training...")
    if torch.cuda.is_available():
        print(f"   • Device: GPU ({torch.cuda.get_device_name(0)})")
    else:
        print("   • Device: CPU/MPS (slow)")
    
    # Save initial training state
    checkpoint_mgr.save_training_state(
        status="training_started",
        num_train_epochs=training_args.num_train_epochs,
        train_samples=len(train_tokenized),
        eval_samples=len(eval_tokenized),
    )
    
    #  Register trainer with shutdown handler for auto-save on interrupt
    shutdown_handler.register_trainer(trainer, model, checkpoint_mgr)
    
    # Train (with optional checkpoint resumption and auto-save on interrupt)
    try:
        trainer.train(resume_from_checkpoint=resume_from_checkpoint)
    except KeyboardInterrupt:
        print("\n  Training interrupted by user (Ctrl+C)")
        print(" Checkpoint already saved by shutdown handler")
        raise

    print("\n Final evaluation:")
    eval_results = trainer.evaluate()
    for key, value in eval_results.items():
        if isinstance(value, (int, float)):
            print(f"   • {key}: {value:.4f}")
        else:
            print(f"   • {key}: {value}")

    # Save adapter (Drive-first)
    adapter_path = str(Path(TRAINING_CONFIG['output_dir']) / "unified_lora_adapter")
    model.save_pretrained(adapter_path)
    tokenizer.save_pretrained(adapter_path)
    print(f"\n Unified LoRA adapter saved to: {adapter_path}")
    
    # Save final training state
    checkpoint_mgr.save_training_state(
        status="training_completed",
        final_eval_loss=eval_results.get('eval_loss'),
        adapter_path=adapter_path,
    )

    return model, trainer

# Train unified adapter (uses pre-split train.jsonl/val.jsonl when available)
# 
# CHECKPOINT OPTIONS:
# - resume_from_checkpoint="auto"  -> Tự động resume từ checkpoint mới nhất
# - resume_from_checkpoint="/path" -> Resume từ checkpoint cụ thể
# - resume_from_checkpoint=None    -> Bắt đầu training mới
#
print("\n" + "="*70)
print(" TRAINING UNIFIED ADAPTER")
print("="*70)
print(f"Resume mode: auto (will auto-detect latest checkpoint)")
print("="*70 + "\n")

unified_model, trainer = finetune_unified_adapter(
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    lora_config=UNIFIED_LORA_CONFIG,
    resume_from_checkpoint="auto",  #  Change to None to start fresh
)

##  Resume Training Sau Khi Đổi Runtime

**Khi nào cần resume:**
-  Colab disconnect hoặc timeout
-  Đổi GPU runtime (T4 → A100, v.v.)
-  Tắt máy/laptop giữa chừng
-  Muốn tiếp tục training từ checkpoint tốt nhất

**Các bước thực hiện:**

### Bước 1: Mount Drive (nếu lưu checkpoint trên Drive)
```python
from google.colab import drive
drive.mount('/content/drive')
```

### Bước 2: Run lại các cell setup (1-3)
- Cell cài đặt packages
- Cell import libraries
- Cell checkpoint manager (để load trạng thái)

### Bước 3: Run cell config và load model/tokenizer

### Bước 4: Load dataset (hoặc dùng dataset đã lưu)

### Bước 5: Resume training
Cell training đã được cấu hình với `resume_from_checkpoint="auto"` nên sẽ tự động:
-  Tìm checkpoint mới nhất
-  Load model weights, optimizer state, scheduler state
-  Tiếp tục từ step đã dừng

**Lưu ý quan trọng:**
-  Đảm bảo config (learning rate, batch size, etc.) giống với lần train trước
-  Checkpoint được lưu mỗi `save_steps=200` steps
-  Chỉ giữ `save_total_limit=3` checkpoints mới nhất để tiết kiệm dung lượng

In [ ]:
#  Utility: Quản lý checkpoints (xóa cũ, chọn checkpoint cụ thể)

def list_checkpoints():
    """Liệt kê tất cả checkpoints với thông tin chi tiết"""
    checkpoints = checkpoint_mgr.list_all_checkpoints()
    
    if not checkpoints:
        print("  Không có checkpoint nào")
        return []
    
    print(f"\n Có {len(checkpoints)} checkpoint(s):\n")
    for i, cp in enumerate(checkpoints, 1):
        size = sum(f.stat().st_size for f in Path(cp['path']).rglob('*') if f.is_file())
        size_mb = size / (1024 * 1024)
        print(f"{i}. Step {cp['step']:,} - {cp['path']}")
        print(f"   Size: {size_mb:.1f} MB\n")
    
    return checkpoints

def remove_checkpoint(checkpoint_path):
    """Xóa một checkpoint cụ thể"""
    import shutil
    path = Path(checkpoint_path)
    if path.exists() and path.is_dir():
        shutil.rmtree(path)
        print(f" Đã xóa: {checkpoint_path}")
    else:
        print(f"  Không tìm thấy: {checkpoint_path}")

def clean_old_checkpoints(keep_last_n=2):
    """Giữ lại n checkpoints mới nhất, xóa các checkpoints cũ"""
    checkpoints = checkpoint_mgr.list_all_checkpoints()
    
    if len(checkpoints) <= keep_last_n:
        print(f"ℹ  Chỉ có {len(checkpoints)} checkpoint(s), không cần cleanup")
        return
    
    to_remove = checkpoints[:-keep_last_n]
    print(f"  Sẽ xóa {len(to_remove)} checkpoint(s) cũ, giữ lại {keep_last_n} checkpoint mới nhất\n")
    
    for cp in to_remove:
        remove_checkpoint(cp['path'])
    
    print(f"\n Cleanup hoàn tất!")

# List checkpoints hiện có
list_checkpoints()

# Uncomment dòng dưới để cleanup (giữ lại 2 checkpoints mới nhất)
# clean_old_checkpoints(keep_last_n=2)

##  Quick Reference: Checkpoint Commands

**Kiểm tra trạng thái:**
```python
checkpoint_mgr.print_status()
```

**List tất cả checkpoints:**
```python
list_checkpoints()
```

**Resume tự động:**
```python
resume_from_checkpoint="auto"  # trong finetune_unified_adapter()
```

**Resume từ checkpoint cụ thể:**
```python
resume_from_checkpoint="/content/drive/MyDrive/LexiLingo/models/unified_adapter/checkpoint-500"
```

**Cleanup checkpoints cũ (giữ 2 mới nhất):**
```python
clean_old_checkpoints(keep_last_n=2)
```

**Xóa checkpoint cụ thể:**
```python
remove_checkpoint("/path/to/checkpoint-1000")
```

## VERIFY: Kiểm tra checkpoint đã lưu vào Drive

RUN CELL BÊN DƯỚI sau khi training xong để verify checkpoint đã được lưu đúng vào Google Drive!

In [ ]:
# VERIFY: Checkpoint đã được lưu vào Drive chưa?
import os
from pathlib import Path

def verify_checkpoint_location():
    """Kiểm tra và hiển thị vị trí checkpoint đã lưu"""
    print("\n" + "="*70)
    print("CHECKPOINT VERIFICATION")
    print("="*70)
    
    output_dir = Path(TRAINING_CONFIG['output_dir'])
    drive_path = Path("/content/drive/MyDrive/LexiLingo/unified_model")
    
    print(f"\n1. Output directory configured:")
    print(f"   {output_dir}")
    print(f"   Resolved: {output_dir.resolve()}")
    
    print(f"\n2. Directory exists: {'YES' if output_dir.exists() else 'NO'}")
    
    if output_dir.exists():
        # List contents
        contents = list(output_dir.iterdir())
        print(f"\n3. Contents ({len(contents)} items):")
        
        checkpoints = [f for f in contents if f.is_dir() and f.name.startswith('checkpoint-')]
        adapters = [f for f in contents if f.is_dir() and 'adapter' in f.name.lower()]
        other_files = [f for f in contents if f.is_file()]
        
        if checkpoints:
            print(f"\n   Checkpoints found: {len(checkpoints)}")
            for cp in sorted(checkpoints):
                size = sum(f.stat().st_size for f in cp.rglob('*') if f.is_file())
                print(f"   - {cp.name} ({size / (1024**2):.1f} MB)")
        
        if adapters:
            print(f"\n   Adapters found: {len(adapters)}")
            for ad in adapters:
                size = sum(f.stat().st_size for f in ad.rglob('*') if f.is_file())
                print(f"   - {ad.name} ({size / (1024**2):.1f} MB)")
        
        if other_files:
            print(f"\n   Other files: {len(other_files)}")
            for f in other_files:
                print(f"   - {f.name} ({f.stat().st_size / 1024:.1f} KB)")
        
        # Check if on Drive
        if str(drive_path) in str(output_dir):
            print(f"\n4. Storage location: GOOGLE DRIVE")
            print(f"   Data is PERSISTENT and safe from Colab disconnects!")
            
            # Provide Drive link
            relative_path = str(output_dir).replace('/content/drive/MyDrive/', '')
            print(f"\n5. Access from Drive:")
            print(f"   Go to: My Drive > {relative_path}")
        else:
            print(f"\n4. Storage location: LOCAL (/content/)")
            print(f"   WARNING: Will be DELETED when Colab session ends!")
            print(f"\n   To save to Drive:")
            print(f"   1. Mount Drive (run Drive mount cell)")
            print(f"   2. Re-run training cell")
    else:
        print(f"\n   ERROR: Output directory not found!")
        print(f"   Training may not have started or failed.")
    
    print("="*70 + "\n")

# Run verification
verify_checkpoint_location()

## 7. Test Inference

## 7. Visualization - Training Metrics

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import pandas as pd

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print(" Visualization libraries loaded")
print(" Available visualizations:")
print("  1. Training loss curves")
print("  2. Task distribution")
print("  3. Model architecture summary")
print("  4. Parameter comparison")
print("  5. Evaluation metrics dashboard")

In [ ]:
# Visualize 1: Training Loss Curves
def plot_training_loss(trainer):
    """Plot training and validation loss over time"""
    if not hasattr(trainer, 'state') or not trainer.state.log_history:
        print("  No training history available. Train the model first!")
        return
    
    # Extract loss values
    train_loss = []
    eval_loss = []
    steps = []
    eval_steps = []
    
    for log in trainer.state.log_history:
        if 'loss' in log:
            train_loss.append(log['loss'])
            steps.append(log['step'])
        if 'eval_loss' in log:
            eval_loss.append(log['eval_loss'])
            eval_steps.append(log['step'])
    
    # Create plot
    fig, ax = plt.subplots(1, 1, figsize=(12, 6))
    
    # Plot training loss
    ax.plot(steps, train_loss, 'b-', linewidth=2, label='Training Loss', alpha=0.8)
    
    # Plot evaluation loss
    if eval_loss:
        ax.plot(eval_steps, eval_loss, 'r-', linewidth=2, label='Validation Loss', alpha=0.8)
    
    ax.set_xlabel('Training Steps', fontsize=12, fontweight='bold')
    ax.set_ylabel('Loss', fontsize=12, fontweight='bold')
    ax.set_title('Training Progress - Unified LoRA Adapter', fontsize=14, fontweight='bold')
    ax.legend(fontsize=11)
    ax.grid(True, alpha=0.3)
    
    # Add min loss annotation
    if train_loss:
        min_loss = min(train_loss)
        min_step = steps[train_loss.index(min_loss)]
        ax.axhline(y=min_loss, color='g', linestyle='--', alpha=0.5, label=f'Min Loss: {min_loss:.4f}')
        ax.annotate(f'Min: {min_loss:.4f}\nStep: {min_step}', 
                   xy=(min_step, min_loss), 
                   xytext=(10, 10), 
                   textcoords='offset points',
                   bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                   fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("\n Training Statistics:")
    print(f"  Total steps: {steps[-1] if steps else 0}")
    print(f"  Initial loss: {train_loss[0]:.4f}" if train_loss else "  N/A")
    print(f"  Final loss: {train_loss[-1]:.4f}" if train_loss else "  N/A")
    print(f"  Min loss: {min(train_loss):.4f}" if train_loss else "  N/A")
    print(f"  Loss reduction: {((train_loss[0] - train_loss[-1]) / train_loss[0] * 100):.1f}%" if len(train_loss) > 1 else "  N/A")

# Example usage (after training)
# plot_training_loss(trainer)

In [ ]:
# Visualize 2: Task Distribution
def plot_task_distribution(dataset):
    """Visualize distribution of tasks in training dataset"""
    # Count tasks
    task_counts = {}
    for item in dataset:
        task = item.get('task', 'unknown')
        task_counts[task] = task_counts.get(task, 0) + 1
    
    # Create figure with 2 subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Pie chart
    colors = ['#4285F4', '#34A853', '#FBBC04', '#EA4335']
    ax1.pie(task_counts.values(), 
            labels=[f'{k.capitalize()}\n({v} samples)' for k, v in task_counts.items()],
            colors=colors,
            autopct='%1.1f%%',
            startangle=90,
            textprops={'fontsize': 11, 'fontweight': 'bold'})
    ax1.set_title('Task Distribution (Pie Chart)', fontsize=14, fontweight='bold')
    
    # Bar chart
    tasks = list(task_counts.keys())
    counts = list(task_counts.values())
    bars = ax2.bar(tasks, counts, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
    
    # Add value labels on bars
    for i, (bar, count) in enumerate(zip(bars, counts)):
        height = bar.get_height()
        ax2.text(bar.get_x() + bar.get_width()/2., height,
                f'{count}\n({count/sum(counts)*100:.1f}%)',
                ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    ax2.set_xlabel('Task Type', fontsize=12, fontweight='bold')
    ax2.set_ylabel('Number of Samples', fontsize=12, fontweight='bold')
    ax2.set_title('Task Distribution (Bar Chart)', fontsize=14, fontweight='bold')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Capitalize x-labels
    ax2.set_xticklabels([t.capitalize() for t in tasks])
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    total = sum(task_counts.values())
    print("\n Dataset Composition:")
    print(f"  Total samples: {total}")
    for task, count in sorted(task_counts.items()):
        print(f"  {task.capitalize()}: {count} ({count/total*100:.1f}%)")
    
    # Check balance
    if len(set(task_counts.values())) == 1:
        print("\n Dataset is perfectly balanced!")
    else:
        max_count = max(task_counts.values())
        min_count = min(task_counts.values())
        ratio = max_count / min_count
        print(f"\n  Imbalance ratio: {ratio:.2f}x (max/min)")
        if ratio > 2:
            print("   Consider balancing tasks for better multi-task learning")

# Example usage
plot_task_distribution(unified_dataset)

In [ ]:
# Visualize 3: Model Architecture & Parameters
def plot_model_architecture():
    """Visualize LoRA configuration and parameter distribution"""
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. LoRA Configuration
    config_data = {
        'Rank (r)': UNIFIED_LORA_CONFIG['r'],
        'Alpha (α)': UNIFIED_LORA_CONFIG['lora_alpha'],
        'Dropout': UNIFIED_LORA_CONFIG['lora_dropout'] * 100,
        'Target Modules': len(UNIFIED_LORA_CONFIG['target_modules'])
    }
    
    ax1.barh(list(config_data.keys()), list(config_data.values()), 
             color=['#4285F4', '#34A853', '#FBBC04', '#EA4335'], alpha=0.8, edgecolor='black')
    ax1.set_xlabel('Value', fontsize=11, fontweight='bold')
    ax1.set_title('LoRA Configuration', fontsize=13, fontweight='bold')
    ax1.grid(True, alpha=0.3, axis='x')
    
    # Add value labels
    for i, (k, v) in enumerate(config_data.items()):
        ax1.text(v, i, f' {v:.1f}' if 'Dropout' in k else f' {int(v)}', 
                va='center', fontweight='bold', fontsize=10)
    
    # 2. Target Modules
    target_modules = UNIFIED_LORA_CONFIG['target_modules']
    module_colors = plt.cm.Set3(np.linspace(0, 1, len(target_modules)))
    
    ax2.barh(range(len(target_modules)), [1]*len(target_modules), 
             color=module_colors, edgecolor='black', alpha=0.8)
    ax2.set_yticks(range(len(target_modules)))
    ax2.set_yticklabels(target_modules, fontsize=10)
    ax2.set_xlabel('Module Enabled', fontsize=11, fontweight='bold')
    ax2.set_title('Target Modules (LoRA Applied)', fontsize=13, fontweight='bold')
    ax2.set_xlim([0, 1.2])
    ax2.grid(False)
    
    # 3. Parameter Comparison
    base_params = 1500  # Million parameters
    lora_params = 45    # Million trainable params
    frozen_params = base_params - lora_params
    
    params_data = {
        'Frozen\nParameters': frozen_params,
        'Trainable\nLoRA Parameters': lora_params
    }
    
    bars = ax3.bar(params_data.keys(), params_data.values(), 
                   color=['#E8EAED', '#4285F4'], alpha=0.8, edgecolor='black', linewidth=2)
    ax3.set_ylabel('Parameters (Million)', fontsize=11, fontweight='bold')
    ax3.set_title('Parameter Distribution', fontsize=13, fontweight='bold')
    ax3.grid(True, alpha=0.3, axis='y')
    
    # Add percentage labels
    total = sum(params_data.values())
    for bar, value in zip(bars, params_data.values()):
        height = bar.get_height()
        percentage = (value / total) * 100
        ax3.text(bar.get_x() + bar.get_width()/2., height,
                f'{int(value)}M\n({percentage:.1f}%)',
                ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # 4. Unified vs Separate Adapters Comparison
    metrics = ['Storage\n(MB)', 'Latency\n(ms)', 'Load Time\n(s)', 'Memory\n(GB)']
    unified_values = [80, 125, 0.8, 2.5]
    separate_values = [320, 500, 4.0, 3.5]
    
    x = np.arange(len(metrics))
    width = 0.35
    
    bars1 = ax4.bar(x - width/2, unified_values, width, label='Unified Adapter',
                    color='#34A853', alpha=0.8, edgecolor='black')
    bars2 = ax4.bar(x + width/2, separate_values, width, label='4 Separate Adapters',
                    color='#EA4335', alpha=0.8, edgecolor='black')
    
    ax4.set_ylabel('Value', fontsize=11, fontweight='bold')
    ax4.set_title('Unified vs Separate Adapters', fontsize=13, fontweight='bold')
    ax4.set_xticks(x)
    ax4.set_xticklabels(metrics, fontsize=10)
    ax4.legend(fontsize=10, loc='upper left')
    ax4.grid(True, alpha=0.3, axis='y')
    
    # Add improvement percentages
    for i, (u, s) in enumerate(zip(unified_values, separate_values)):
        improvement = ((s - u) / s) * 100
        ax4.text(i, max(u, s) + 20, f'↓{improvement:.0f}%',
                ha='center', fontweight='bold', fontsize=9, color='green')
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n Model Architecture Summary:")
    print(f"  Base Model: Qwen2.5-1.5B-Instruct")
    print(f"  Total Parameters: {base_params}M")
    print(f"  Trainable Parameters: {lora_params}M ({(lora_params/base_params)*100:.2f}%)")
    print(f"  LoRA Rank: {UNIFIED_LORA_CONFIG['r']}")
    print(f"  LoRA Alpha: {UNIFIED_LORA_CONFIG['lora_alpha']}")
    print(f"  Target Modules: {len(UNIFIED_LORA_CONFIG['target_modules'])}")
    print(f"\n Unified Adapter Advantages:")
    print(f"  • 75% smaller storage (80MB vs 320MB)")
    print(f"  • 75% faster inference (125ms vs 500ms)")
    print(f"  • 80% faster loading (0.8s vs 4s)")
    print(f"  • 29% less memory (2.5GB vs 3.5GB)")

# Example usage
plot_model_architecture()

In [ ]:
# Visualize 4: Evaluation Metrics Dashboard
def plot_evaluation_dashboard(eval_results):
    """
    Comprehensive dashboard for evaluation metrics
    
    Args:
        eval_results: dict with keys 'fluency', 'vocabulary', 'grammar', 'dialogue'
                     Each containing task-specific metrics
    """
    
    fig = plt.figure(figsize=(18, 10))
    gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
    
    # 1. Overall Task Performance (Top Left)
    ax1 = fig.add_subplot(gs[0, 0])
    tasks = list(eval_results.keys())
    # Normalize scores to 0-100 scale
    scores = []
    for task in tasks:
        if task == 'fluency':
            # MAE: lower is better, normalize inversely (assume max MAE = 2.0)
            mae = eval_results[task].get('mae', 1.0)
            scores.append((1 - min(mae/2.0, 1.0)) * 100)
        elif task == 'vocabulary':
            # Accuracy: 0-100
            scores.append(eval_results[task].get('accuracy', 0) * 100)
        elif task == 'grammar':
            # F0.5: 0-100
            scores.append(eval_results[task].get('f0.5', 0) * 100)
        elif task == 'dialogue':
            # Quality score: 0-100
            scores.append(eval_results[task].get('avg_quality', 0) * 100)
    
    colors = ['#4285F4', '#34A853', '#FBBC04', '#EA4335']
    bars = ax1.barh(tasks, scores, color=colors, alpha=0.8, edgecolor='black')
    ax1.set_xlabel('Performance Score (0-100)', fontsize=10, fontweight='bold')
    ax1.set_title('Overall Task Performance', fontsize=12, fontweight='bold')
    ax1.set_xlim([0, 100])
    ax1.grid(True, alpha=0.3, axis='x')
    
    for bar, score in zip(bars, scores):
        ax1.text(score + 2, bar.get_y() + bar.get_height()/2, 
                f'{score:.1f}', va='center', fontweight='bold', fontsize=9)
    
    # 2. Fluency Metrics (Top Middle)
    ax2 = fig.add_subplot(gs[0, 1])
    if 'fluency' in eval_results:
        fluency = eval_results['fluency']
        metrics = ['MAE', 'MSE', 'Pearson r']
        values = [
            fluency.get('mae', 0),
            fluency.get('mse', 0),
            fluency.get('pearson_r', 0)
        ]
        bars = ax2.bar(metrics, values, color=['#EA4335', '#FBBC04', '#34A853'], 
                      alpha=0.8, edgecolor='black')
        ax2.set_ylabel('Value', fontsize=10, fontweight='bold')
        ax2.set_title('Fluency Metrics', fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars, values):
            ax2.text(bar.get_x() + bar.get_width()/2, val,
                    f'{val:.3f}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 3. Vocabulary Metrics (Top Right)
    ax3 = fig.add_subplot(gs[0, 2])
    if 'vocabulary' in eval_results:
        vocab = eval_results['vocabulary']
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1']
        values = [
            vocab.get('accuracy', 0) * 100,
            vocab.get('precision', 0) * 100,
            vocab.get('recall', 0) * 100,
            vocab.get('f1', 0) * 100
        ]
        bars = ax3.bar(metrics, values, color='#4285F4', alpha=0.8, edgecolor='black')
        ax3.set_ylabel('Score (%)', fontsize=10, fontweight='bold')
        ax3.set_title('Vocabulary Metrics', fontsize=12, fontweight='bold')
        ax3.set_ylim([0, 100])
        ax3.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars, values):
            ax3.text(bar.get_x() + bar.get_width()/2, val,
                    f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 4. Grammar Metrics (Middle Left)
    ax4 = fig.add_subplot(gs[1, 0])
    if 'grammar' in eval_results:
        grammar = eval_results['grammar']
        metrics = ['Precision', 'Recall', 'F0.5', 'F1']
        values = [
            grammar.get('precision', 0) * 100,
            grammar.get('recall', 0) * 100,
            grammar.get('f0.5', 0) * 100,
            grammar.get('f1', 0) * 100
        ]
        bars = ax4.bar(metrics, values, color='#FBBC04', alpha=0.8, edgecolor='black')
        ax4.set_ylabel('Score (%)', fontsize=10, fontweight='bold')
        ax4.set_title('Grammar Correction Metrics', fontsize=12, fontweight='bold')
        ax4.set_ylim([0, 100])
        ax4.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars, values):
            ax4.text(bar.get_x() + bar.get_width()/2, val,
                    f'{val:.1f}%', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 5. Dialogue Quality Distribution (Middle Center)
    ax5 = fig.add_subplot(gs[1, 1])
    if 'dialogue' in eval_results:
        dialogue = eval_results['dialogue']
        quality_dist = dialogue.get('quality_distribution', {1: 0, 2: 0, 3: 0, 4: 0, 5: 0})
        
        levels = list(quality_dist.keys())
        counts = list(quality_dist.values())
        colors_qual = ['#EA4335', '#FBBC04', '#F4B400', '#34A853', '#0F9D58']
        
        bars = ax5.bar(levels, counts, color=colors_qual[:len(levels)], 
                      alpha=0.8, edgecolor='black')
        ax5.set_xlabel('Quality Level', fontsize=10, fontweight='bold')
        ax5.set_ylabel('Count', fontsize=10, fontweight='bold')
        ax5.set_title('Dialogue Quality Distribution', fontsize=12, fontweight='bold')
        ax5.set_xticks(levels)
        ax5.grid(True, alpha=0.3, axis='y')
        
        for bar, count in zip(bars, counts):
            if count > 0:
                ax5.text(bar.get_x() + bar.get_width()/2, count,
                        f'{count}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 6. Task-wise Sample Counts (Middle Right)
    ax6 = fig.add_subplot(gs[1, 2])
    task_samples = {task: eval_results[task].get('total_samples', 0) 
                    for task in tasks if task in eval_results}
    
    bars = ax6.bar(task_samples.keys(), task_samples.values(), 
                   color=colors[:len(task_samples)], alpha=0.8, edgecolor='black')
    ax6.set_ylabel('Number of Samples', fontsize=10, fontweight='bold')
    ax6.set_title('Evaluation Dataset Size', fontsize=12, fontweight='bold')
    ax6.grid(True, alpha=0.3, axis='y')
    ax6.tick_params(axis='x', rotation=45)
    
    for bar, count in zip(bars, task_samples.values()):
        ax6.text(bar.get_x() + bar.get_width()/2, count,
                f'{count}', ha='center', va='bottom', fontweight='bold', fontsize=9)
    
    # 7. Performance vs Target (Bottom Span)
    ax7 = fig.add_subplot(gs[2, :])
    
    # Define targets from architecture.md
    targets = {
        'fluency': {'metric': 'MAE', 'target': 0.5, 'current': eval_results.get('fluency', {}).get('mae', 1.0), 'better': 'lower'},
        'vocabulary': {'metric': 'Accuracy', 'target': 85, 'current': eval_results.get('vocabulary', {}).get('accuracy', 0) * 100, 'better': 'higher'},
        'grammar': {'metric': 'F0.5', 'target': 60, 'current': eval_results.get('grammar', {}).get('f0.5', 0) * 100, 'better': 'higher'},
        'dialogue': {'metric': 'Quality', 'target': 4.0, 'current': eval_results.get('dialogue', {}).get('avg_quality', 0) * 20, 'better': 'higher'}  # Scale 1-5 to 0-100
    }
    
    x_pos = np.arange(len(targets))
    width = 0.35
    
    target_vals = [targets[task]['target'] for task in tasks]
    current_vals = [targets[task]['current'] for task in tasks]
    
    bars1 = ax7.bar(x_pos - width/2, target_vals, width, label='Target',
                    color='#9AA0A6', alpha=0.8, edgecolor='black')
    bars2 = ax7.bar(x_pos + width/2, current_vals, width, label='Current',
                    color=colors, alpha=0.8, edgecolor='black')
    
    ax7.set_ylabel('Value', fontsize=11, fontweight='bold')
    ax7.set_title('Current Performance vs Target Metrics', fontsize=13, fontweight='bold')
    ax7.set_xticks(x_pos)
    ax7.set_xticklabels([f"{task.title()}\n({targets[task]['metric']})" for task in tasks], fontsize=10)
    ax7.legend(fontsize=11, loc='upper left')
    ax7.grid(True, alpha=0.3, axis='y')
    
    # Add achievement status
    for i, task in enumerate(tasks):
        target = targets[task]['target']
        current = targets[task]['current']
        better = targets[task]['better']
        
        if better == 'lower':
            achieved = current <= target
            symbol = '' if achieved else ''
            color = 'green' if achieved else 'red'
        else:
            achieved = current >= target
            symbol = '' if achieved else ''
            color = 'green' if achieved else 'red'
        
        ax7.text(i, max(target, current) + 5, symbol,
                ha='center', fontweight='bold', fontsize=16, color=color)
    
    plt.suptitle('Evaluation Metrics Dashboard - Unified LoRA Adapter', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.show()
    
    # Print detailed summary
    print("\n Evaluation Results Summary:")
    print("=" * 60)
    for task in tasks:
        print(f"\n{task.upper()}:")
        target_info = targets[task]
        print(f"  Metric: {target_info['metric']}")
        print(f"  Target: {target_info['target']}")
        print(f"  Current: {target_info['current']:.2f}")
        
        if target_info['better'] == 'lower':
            achieved = target_info['current'] <= target_info['target']
            diff = target_info['target'] - target_info['current']
        else:
            achieved = target_info['current'] >= target_info['target']
            diff = target_info['current'] - target_info['target']
        
        status = " ACHIEVED" if achieved else " NOT ACHIEVED"
        print(f"  Status: {status}")
        print(f"  Difference: {abs(diff):.2f} ({'+' if diff > 0 else ''}{diff:.2f})")
        
        # Additional metrics
        if task in eval_results:
            for key, value in eval_results[task].items():
                if key not in ['total_samples', 'quality_distribution']:
                    print(f"  {key}: {value if isinstance(value, int) else f'{value:.4f}'}")

# Example usage with mock data
eval_results_example = {
    'fluency': {'mae': 0.45, 'mse': 0.32, 'pearson_r': 0.87, 'total_samples': 500},
    'vocabulary': {'accuracy': 0.88, 'precision': 0.85, 'recall': 0.90, 'f1': 0.87, 'total_samples': 300},
    'grammar': {'precision': 0.72, 'recall': 0.55, 'f0.5': 0.68, 'f1': 0.62, 'total_samples': 400},
    'dialogue': {
        'avg_quality': 4.2,
        'quality_distribution': {1: 5, 2: 15, 3: 80, 4: 120, 5: 80},
        'total_samples': 300
    }
}

# Uncomment to test:
# plot_evaluation_dashboard(eval_results_example)

In [ ]:
# Visualize 5: Training Progress Timeline
def plot_training_timeline(trainer, save_path='training_timeline.png'):
    """
    Visualize complete training timeline with all metrics
    
    Args:
        trainer: Hugging Face Trainer object after training
        save_path: Path to save the plot
    """
    
    if not hasattr(trainer.state, 'log_history') or not trainer.state.log_history:
        print(" No training history available!")
        return
    
    log_history = trainer.state.log_history
    
    # Extract metrics
    steps = []
    train_loss = []
    eval_loss = []
    learning_rates = []
    epochs = []
    
    for entry in log_history:
        if 'loss' in entry:  # Training step
            steps.append(entry.get('step', 0))
            train_loss.append(entry['loss'])
            learning_rates.append(entry.get('learning_rate', 0))
            epochs.append(entry.get('epoch', 0))
        elif 'eval_loss' in entry:  # Evaluation step
            eval_loss.append((entry.get('step', 0), entry['eval_loss']))
    
    # Create figure with 4 subplots
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 12))
    
    # 1. Training & Validation Loss Over Time
    ax1.plot(steps, train_loss, label='Training Loss', color='#4285F4', linewidth=2, alpha=0.8)
    
    if eval_loss:
        eval_steps, eval_losses = zip(*eval_loss)
        ax1.plot(eval_steps, eval_losses, label='Validation Loss', 
                color='#EA4335', linewidth=2, marker='o', markersize=6, alpha=0.8)
        
        # Mark best validation loss
        best_idx = np.argmin(eval_losses)
        best_step = eval_steps[best_idx]
        best_loss = eval_losses[best_idx]
        ax1.plot(best_step, best_loss, marker='*', markersize=20, 
                color='#34A853', label=f'Best Val Loss: {best_loss:.4f}')
        ax1.annotate(f'Best: {best_loss:.4f}', 
                    xy=(best_step, best_loss), 
                    xytext=(best_step, best_loss + 0.1),
                    arrowprops=dict(arrowstyle='->', color='#34A853', lw=2),
                    fontsize=10, fontweight='bold', color='#34A853')
    
    ax1.set_xlabel('Training Steps', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Loss', fontsize=11, fontweight='bold')
    ax1.set_title('Training & Validation Loss Timeline', fontsize=13, fontweight='bold')
    ax1.legend(fontsize=10, loc='upper right')
    ax1.grid(True, alpha=0.3)
    
    # 2. Learning Rate Schedule
    ax2.plot(steps, learning_rates, color='#FBBC04', linewidth=2, alpha=0.8)
    ax2.set_xlabel('Training Steps', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Learning Rate', fontsize=11, fontweight='bold')
    ax2.set_title('Learning Rate Schedule', fontsize=13, fontweight='bold')
    ax2.grid(True, alpha=0.3)
    ax2.ticklabel_format(style='scientific', axis='y', scilimits=(0,0))
    
    # Add annotations for LR changes
    if len(learning_rates) > 1:
        # Find warmup end (where LR stops increasing)
        lr_diffs = np.diff(learning_rates)
        warmup_end = np.where(lr_diffs < 0)[0]
        if len(warmup_end) > 0:
            warmup_step = steps[warmup_end[0]]
            warmup_lr = learning_rates[warmup_end[0]]
            ax2.axvline(x=warmup_step, color='red', linestyle='--', alpha=0.5)
            ax2.text(warmup_step, warmup_lr, f' Warmup End\n Step {warmup_step}',
                    fontsize=9, color='red', fontweight='bold')
    
    # 3. Loss Improvement Rate (gradient)
    if len(train_loss) > 10:
        window_size = max(10, len(train_loss) // 20)
        smoothed_loss = np.convolve(train_loss, np.ones(window_size)/window_size, mode='valid')
        smoothed_steps = steps[:len(smoothed_loss)]
        
        # Calculate gradient (improvement rate)
        gradients = np.gradient(smoothed_loss)
        
        colors = ['#34A853' if g < 0 else '#EA4335' for g in gradients]
        ax3.bar(smoothed_steps, gradients, color=colors, alpha=0.6, width=max(1, len(steps)//50))
        ax3.axhline(y=0, color='black', linestyle='-', linewidth=1)
        ax3.set_xlabel('Training Steps', fontsize=11, fontweight='bold')
        ax3.set_ylabel('Loss Change Rate', fontsize=11, fontweight='bold')
        ax3.set_title('Training Progress Rate (Green=Improving, Red=Worsening)', 
                     fontsize=13, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='y')
        
        # Add statistics
        avg_improvement = np.mean([g for g in gradients if g < 0])
        ax3.text(0.02, 0.98, f'Avg Improvement Rate: {avg_improvement:.6f}',
                transform=ax3.transAxes, fontsize=10, fontweight='bold',
                verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    # 4. Training Statistics Summary
    ax4.axis('off')
    
    # Calculate statistics
    total_steps = len(steps)
    total_epochs = epochs[-1] if epochs else 0
    initial_loss = train_loss[0] if train_loss else 0
    final_loss = train_loss[-1] if train_loss else 0
    best_train_loss = min(train_loss) if train_loss else 0
    loss_reduction = ((initial_loss - final_loss) / initial_loss * 100) if initial_loss > 0 else 0
    
    # Best validation metrics
    if eval_loss:
        best_val_loss = min([l for _, l in eval_loss])
        best_val_step = [s for s, l in eval_loss if l == best_val_loss][0]
    else:
        best_val_loss = None
        best_val_step = None
    
    # Training speed
    if len(steps) > 1:
        avg_steps_per_unit = (steps[-1] - steps[0]) / (len(steps) - 1)
    else:
        avg_steps_per_unit = 0
    
    # Create summary text
    summary_text = f"""
     TRAINING SUMMARY
    {'='*50}
    
    Training Progress:
      • Total Steps: {total_steps:,}
      • Total Epochs: {total_epochs:.2f}
      • Avg Steps/Update: {avg_steps_per_unit:.1f}
    
    Loss Metrics:
      • Initial Loss: {initial_loss:.4f}
      • Final Loss: {final_loss:.4f}
      • Best Train Loss: {best_train_loss:.4f}
      • Loss Reduction: {loss_reduction:.2f}%
    
    """
    
    if best_val_loss is not None:
        summary_text += f"""
    Validation:
      • Best Val Loss: {best_val_loss:.4f}
      • Best Val Step: {best_val_step:,}
    
    """
    
    summary_text += f"""
    Learning Rate:
      • Initial LR: {learning_rates[0]:.2e}
      • Final LR: {learning_rates[-1]:.2e}
      • Max LR: {max(learning_rates):.2e}
    
    """
    
    # Convergence status
    if len(train_loss) > 50:
        recent_variance = np.var(train_loss[-50:])
        early_variance = np.var(train_loss[:50])
        convergence_ratio = recent_variance / early_variance if early_variance > 0 else 0
        
        if convergence_ratio < 0.1:
            convergence_status = " CONVERGED"
            convergence_color = '#34A853'
        elif convergence_ratio < 0.5:
            convergence_status = " CONVERGING"
            convergence_color = '#FBBC04'
        else:
            convergence_status = " NOT CONVERGED"
            convergence_color = '#EA4335'
        
        summary_text += f"""
    Convergence:
      • Status: {convergence_status}
      • Variance Ratio: {convergence_ratio:.4f}
    """
    
    ax4.text(0.1, 0.95, summary_text, transform=ax4.transAxes,
            fontsize=11, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='#E8EAED', alpha=0.8, pad=1))
    
    plt.suptitle(f'Training Timeline - Unified LoRA Adapter', 
                 fontsize=16, fontweight='bold', y=0.995)
    plt.tight_layout()
    
    # Save plot
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    print(f" Training timeline saved to: {save_path}")
    
    plt.show()
    
    # Print console summary
    print("\n" + summary_text)

# Example usage after training:
# plot_training_timeline(trainer)

## 8. Usage Examples - Complete Training Pipeline

Below are examples showing how to use all components together:

In [ ]:
# Complete Training & Evaluation Pipeline with Visualization

# Step 1: Setup MongoDB Logger (Optional)
mongo_logger = MongoDBLogger(
    uri="mongodb://localhost:27017/",
    database="lexilingo"
)

# Step 2: Visualize dataset before training
print(" Dataset Overview:")
plot_task_distribution(unified_training_data)
print("\n" + "="*60 + "\n")

# Step 3: Visualize model architecture
print(" Model Architecture:")
plot_model_architecture()
print("\n" + "="*60 + "\n")

# Step 4: Fine-tune the unified adapter
print(" Starting Training...")
trainer, model, tokenizer = finetune_unified_adapter(
    training_data=unified_training_data,
    output_dir="./model/outputs/unified_adapter",
    num_epochs=3,
    batch_size=4,
    learning_rate=2e-4,
    mongo_logger=mongo_logger  # Optional: None to disable logging
)
print(" Training completed!")
print("\n" + "="*60 + "\n")

# Step 5: Visualize training progress
print(" Training Progress:")
plot_training_loss(trainer)
plot_training_timeline(trainer, save_path='./model/outputs/unified_adapter/training_timeline.png')
print("\n" + "="*60 + "\n")

# Step 6: Test the model
print(" Testing Unified Adapter:")
test_unified_adapter(model, tokenizer)
print("\n" + "="*60 + "\n")

# Step 7: Comprehensive evaluation (with separate test set)
print(" Running Comprehensive Evaluation...")

# Example test sets (replace with actual test data)
test_datasets = {
    'fluency': [
        {"input": "Sample text", "label": 4.5},
        # ... more test samples
    ],
    'vocabulary': [
        {"input": "Context with word", "label": "correct_word"},
        # ... more test samples
    ],
    'grammar': [
        {"input": "Text with error", "label": "corrected text"},
        # ... more test samples
    ],
    'dialogue': [
        {"input": "User message", "label": "Bot response"},
        # ... more test samples
    ]
}

# Run evaluation for each task
eval_results = {
    'fluency': evaluate_fluency_task(model, tokenizer, test_datasets['fluency']),
    'vocabulary': evaluate_vocabulary_task(model, tokenizer, test_datasets['vocabulary']),
    'grammar': evaluate_grammar_task(model, tokenizer, test_datasets['grammar']),
    'dialogue': evaluate_dialogue_task(model, tokenizer, test_datasets['dialogue'])
}

# Step 8: Visualize evaluation results
print(" Evaluation Dashboard:")
plot_evaluation_dashboard(eval_results)
print("\n" + "="*60 + "\n")

# Step 9: Save the final adapter
final_output_path = "./model/adapters/unified_lora_adapter_v2"
model.save_pretrained(final_output_path)
tokenizer.save_pretrained(final_output_path)
print(f" Unified adapter saved to: {final_output_path}")

print("\n" + "="*80)
print(" Complete Training & Evaluation Pipeline Finished!")
print("="*80)

In [ ]:
from peft import PeftModel

def test_unified_adapter(task, test_input):
    """
    Test unified LoRA adapter for specific task
    
    Args:
        task: Task name (fluency, vocabulary, grammar, dialogue)
        test_input: Test input text
    """
    print(f"\n{'='*60}")
    print(f"Testing UNIFIED adapter - Task: {task.upper()}")
    print(f"{'='*60}\n")
    
    # Load base model + unified adapter
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float32,
        bnb_4bit_use_double_quant=True,
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    
    model = PeftModel.from_pretrained(
        model,
        "./adapters/unified_lora_adapter"
    )
    
    # Create task-specific prompt
    if task == "fluency":
        prompt = f"""Analyze the fluency of this English sentence:
Input: {test_input}

Provide a JSON response with:
- fluency_score (0.0-1.0)
- reasoning (brief explanation)"""
    
    elif task == "vocabulary":
        prompt = f"""Classify the vocabulary level of this English sentence:
Input: {test_input}

Provide a JSON response with:
- level (A2, B1, or B2)
- key_words (important words with their levels)"""
    
    elif task == "grammar":
        prompt = f"""Correct the grammar errors in this English sentence:
Input: {test_input}

Provide a JSON response with:
- corrected (corrected sentence)
- explanation (brief error explanation)"""
    
    elif task == "dialogue":
        prompt = f"""Generate an encouraging tutor response:
Input: {test_input}

Provide a JSON response with:
- response (supportive tutor message)"""
    
    # Prepare input
    messages = [{"role": "user", "content": prompt}]
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    inputs = tokenizer(text, return_tensors="pt")
    
    # Generate
    print(f"Input: {test_input}")
    print(" Generating response...")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=256,
            do_sample=True,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    print(f"\nResponse: {response}")
    print("="*60)
    
    # Try to parse JSON
    try:
        parsed = json.loads(response)
        print(f"\n Valid JSON output:")
        for key, value in parsed.items():
            print(f"  {key}: {value}")
    except:
        print("\n  Response is not valid JSON (model needs more training)")
    
    return response

In [ ]:
# Test all tasks with the same unified adapter
print("Testing UNIFIED adapter across all tasks...")
print("="*60)

# Test 1: Fluency Scoring
test_unified_adapter("fluency", "She plays piano every day")

In [ ]:
# Test 2: Vocabulary Classification
test_unified_adapter("vocabulary", "The presentation was incredibly sophisticated")

In [ ]:
# Test 3: Grammar Correction
test_unified_adapter("grammar", "He don't want to go there")

In [ ]:
# Test 4: Dialogue Generation
test_unified_adapter("dialogue", "I likes playing basketball | fluency:0.60 | level:A2 | errors:Subject-verb agreement")

## 8. Evaluation Metrics (Based on Architecture.md)

In [ ]:
# Evaluation metrics according to architecture.md Section 5.4
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from scipy.stats import pearsonr

def evaluate_fluency_task(predictions, ground_truth):
    """
    Evaluate fluency scoring task
    Target: MAE < 0.12, Pearson > 0.90
    """
    predictions = np.array([p['fluency_score'] for p in predictions])
    ground_truth = np.array([g['fluency_score'] for g in ground_truth])
    
    mae = mean_absolute_error(ground_truth, predictions)
    pearson_corr, _ = pearsonr(predictions, ground_truth)
    
    print(f"Fluency Scoring Metrics:")
    print(f"  MAE: {mae:.4f} (target: < 0.12)")
    print(f"  Pearson Correlation: {pearson_corr:.4f} (target: > 0.90)")
    print(f"  Status: {' PASS' if mae < 0.12 and pearson_corr > 0.90 else ' NEEDS IMPROVEMENT'}")
    
    return {"mae": mae, "pearson": pearson_corr}

def evaluate_vocabulary_task(predictions, ground_truth):
    """
    Evaluate vocabulary classification task
    Target: Accuracy > 90%, Macro F1 > 0.88
    """
    pred_levels = [p['level'] for p in predictions]
    true_levels = [g['level'] for g in ground_truth]
    
    accuracy = accuracy_score(true_levels, pred_levels)
    macro_f1 = f1_score(true_levels, pred_levels, average='macro')
    
    print(f"\nVocabulary Classification Metrics:")
    print(f"  Accuracy: {accuracy:.4f} (target: > 0.90)")
    print(f"  Macro F1: {macro_f1:.4f} (target: > 0.88)")
    print(f"  Status: {' PASS' if accuracy > 0.90 and macro_f1 > 0.88 else ' NEEDS IMPROVEMENT'}")
    
    return {"accuracy": accuracy, "macro_f1": macro_f1}

def evaluate_grammar_task(predictions, ground_truth):
    """
    Evaluate grammar correction task
    Target: F0.5 > 68, Precision > 72, Recall > 60
    """
    # For grammar, we check if correction matches exactly
    pred_corrections = [p['corrected'] for p in predictions]
    true_corrections = [g['corrected'] for g in ground_truth]
    
    # Binary: correct (1) or incorrect (0)
    matches = [1 if p == t else 0 for p, t in zip(pred_corrections, true_corrections)]
    
    # Simplified metrics (in production, use proper TP/FP/FN counting)
    accuracy = sum(matches) / len(matches)
    
    print(f"\nGrammar Correction Metrics:")
    print(f"  Exact Match Accuracy: {accuracy:.4f}")
    print(f"  Note: Use proper F0.5 metric with TP/FP/FN in production")
    print(f"  Target: F0.5 > 0.68, Precision > 0.72, Recall > 0.60")
    
    return {"exact_match": accuracy}

def evaluate_dialogue_task(predictions, ground_truth):
    """
    Evaluate dialogue generation task
    Target: Quality > 96% (human evaluation), Appropriateness > 94%
    """
    # In production, use human evaluation or GPT-4 as judge
    # Here we just check response length as proxy
    pred_responses = [p['response'] for p in predictions]
    
    avg_length = np.mean([len(r.split()) for r in pred_responses])
    
    print(f"\nDialogue Generation Metrics:")
    print(f"  Average Response Length: {avg_length:.1f} words")
    print(f"  Note: Use human evaluation or LLM-as-judge in production")
    print(f"  Target: Quality > 96%, Appropriateness > 94%")
    
    return {"avg_response_length": avg_length}

# Example evaluation (replace with actual test data)
print("="*60)
print("EVALUATION METRICS (Architecture.md Section 5.4)")
print("="*60)

print("\n To run full evaluation:")
print("  1. Prepare test datasets (separate from training)")
print("  2. Run inference on test data")
print("  3. Calculate metrics using functions above")
print("  4. Compare with targets from architecture.md")

print("\n Target Metrics Summary:")
print("  Fluency: MAE < 0.12, Pearson > 0.90")
print("  Vocabulary: Accuracy > 90%, Macro F1 > 0.88")
print("  Grammar: F0.5 > 68, Precision > 72, Recall > 60")
print("  Dialogue: Quality > 96%, Appropriateness > 94%")

## 9. Export for Production

In [ ]:
# Merge LoRA weights into base model (optional for deployment)
def merge_unified_adapter():
    """
    Merge unified LoRA adapter into base model
    Creates standalone model without PEFT dependency
    """
    print("Merging unified adapter into base model...")
    
    # Load base + adapter
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        torch_dtype=torch.float32,  # Use float32 for CPU
        device_map="auto",
        trust_remote_code=True,
        low_cpu_mem_usage=True
    )
    
    model = PeftModel.from_pretrained(
        model,
        "./adapters/unified_lora_adapter"
    )
    
    # Merge and unload
    merged_model = model.merge_and_unload()
    
    # Save merged model
    output_path = "./merged_models/unified_merged"
    Path(output_path).mkdir(parents=True, exist_ok=True)
    
    merged_model.save_pretrained(output_path)
    tokenizer.save_pretrained(output_path)
    
    print(f" Merged model saved to: {output_path}")
    print(f"  Size: ~1.5GB")
    print(f"  Format: HuggingFace Transformers")
    
    return merged_model

# Optional: Merge for deployment
# merged_model = merge_unified_adapter()

print("="*60)
print("EXPORT OPTIONS")
print("="*60)
print("\n1. Keep LoRA Adapter (Recommended):")
print("   - Size: 80MB (adapter only)")
print("   - Load time: <1s")
print("   - Requires: Base model + PEFT library")
print("   - Best for: Development & testing")

print("\n2. Merged Model:")
print("   - Size: 1.5GB (full model)")
print("   - Load time: ~3s")
print("   - Requires: Only Transformers library")
print("   - Best for: Production deployment")

print("\n3. Quantized (Coming soon):")
print("   - Size: 400MB (4-bit quantized)")
print("   - Load time: ~2s")
print("   - Best for: Mobile deployment")